In [ ]:
import pandas as pd
import re

In [139]:
import pandas as pd
import re

def expand_quarter_to_daily(df, quarter_col=None, use_index=False):
    """
    Expands rows in the DataFrame so that each valid quarter is expanded into daily rows
    covering the entire quarter, with the same values copied to each day.

    Expected quarter format: "year number kwartaal" (e.g., "2020 4e kwartaal" or "2020 4 kwartaal").
    Rows are skipped if:
      - The value is exactly "Bron: CBS"
      - The value is not in the expected quarter format (e.g., if it’s only a year or otherwise invalid).

    Parameters:
        df (pd.DataFrame): The original DataFrame.
        quarter_col (str, optional): Name of the column containing the quarter info.
                                     Must be provided if use_index is False.
        use_index (bool): If True, the quarter info is taken from the DataFrame's index.

    Returns:
        pd.DataFrame: A new DataFrame with a daily date index and expanded rows.
    """
    def convert_quarter_string(s):
        s = s.strip()
        # Expected pattern: "YYYY <number>[optional letter] kwartaal"
        pattern = r"^(\d{4})\s+(\d+)[a-zA-Z]*\s+kwartaal$"
        m = re.fullmatch(pattern, s)
        if m:
            year = m.group(1)
            quarter = m.group(2)
            return pd.Period(f"{year}Q{quarter}", freq='Q')
        else:
            return None

    expanded_rows = []
    
    for idx, row in df.iterrows():
        # Extract the quarter string either from the index or the specified column.
        if use_index:
            quarter_str = str(idx).strip()
        else:
            if quarter_col is None:
                raise ValueError("quarter_col must be provided if use_index is False.")
            quarter_str = str(row[quarter_col]).strip()
        
        # Skip rows that should be dropped.
        if quarter_str == "Bron: CBS":
            continue
        
        period = convert_quarter_string(quarter_str)
        if period is None:
            continue  # Skip rows not matching the expected quarter format.
        
        start_date = period.start_time
        end_date = period.end_time
        
        # Create a daily date range for the quarter.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        # Duplicate the row for each day.
        row_expanded = pd.DataFrame([row] * len(daily_index), index=daily_index)
        row_expanded['date'] = daily_index
        
        expanded_rows.append(row_expanded)
    
    return pd.concat(expanded_rows) if expanded_rows else pd.DataFrame()


def expand_year_to_daily(df, year_col=None, use_index=False):
    """
    Expands rows in the DataFrame so that each valid year is expanded into daily rows
    covering the entire year, with the same values copied to each day.

    Expected year format: a four-digit year (e.g., "2020").
    Rows are skipped if:
      - The value is exactly "Bron: CBS"
      - The value is not a valid four-digit year.

    Parameters:
        df (pd.DataFrame): The original DataFrame.
        year_col (str, optional): Name of the column containing the year info.
                                  Must be provided if use_index is False.
        use_index (bool): If True, the year info is taken from the DataFrame's index.

    Returns:
        pd.DataFrame: A new DataFrame with a daily date index and expanded rows.
    """
    def convert_year_string(s):
        s = s.strip()
        # Expected: exactly a four-digit year.
        m = re.fullmatch(r"\d{4}", s)
        if m:
            return int(s)
        else:
            return None

    expanded_rows = []
    
    for idx, row in df.iterrows():
        # Extract the year string either from the index or from the specified column.
        if use_index:
            year_str = str(idx).strip()
        else:
            if year_col is None:
                raise ValueError("year_col must be provided if use_index is False.")
            year_str = str(row[year_col]).strip()
        
        # Skip rows that should be dropped.
        if year_str == "Bron: CBS":
            continue
        
        year_int = convert_year_string(year_str)
        if year_int is None:
            continue  # Skip rows that do not have a valid year.
        
        start_date = pd.Timestamp(year=year_int, month=1, day=1)
        end_date = pd.Timestamp(year=year_int, month=12, day=31)
        
        # Create a daily date range for the year.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        # Duplicate the row for each day.
        row_expanded = pd.DataFrame([row] * len(daily_index), index=daily_index)
        row_expanded['date'] = daily_index
        
        expanded_rows.append(row_expanded)
    
    return pd.concat(expanded_rows) if expanded_rows else pd.DataFrame()


# Example usage:
# Quarter-based expansion using a column:
data_quarter = {
    'quarter': ["2020 4e kwartaal   ", "2021 1e kwartaal", "2022", "Bron: CBS", "2023 extra"],
    'value': [100, 200, 300, 400, 500]
}
df_quarter = pd.DataFrame(data_quarter)
df_quarter_expanded = expand_quarter_to_daily(df_quarter, quarter_col='quarter')
print("Quarter-based expansion (using column):")
print(df_quarter_expanded.head())

# Year-based expansion using the index:
data_year = {
    'value': [100, 200, 300, 400]
}
# Create a DataFrame with the index as year strings.
df_year = pd.DataFrame(data_year, index=["2020", "2021", "Bron: CBS", "2023"])
df_year_expanded = expand_year_to_daily(df_year, use_index=True)
print("\nYear-based expansion (using index):")
print(df_year_expanded.head())


Quarter-based expansion (using column):
                        quarter  value       date
2020-10-01  2020 4e kwartaal       100 2020-10-01
2020-10-02  2020 4e kwartaal       100 2020-10-02
2020-10-03  2020 4e kwartaal       100 2020-10-03
2020-10-04  2020 4e kwartaal       100 2020-10-04
2020-10-05  2020 4e kwartaal       100 2020-10-05

Year-based expansion (using index):
            value       date
2020-01-01    100 2020-01-01
2020-01-02    100 2020-01-02
2020-01-03    100 2020-01-03
2020-01-04    100 2020-01-04
2020-01-05    100 2020-01-05


In [140]:
LaborParticipation = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Arbeidsdeelname__kerncijfers_seizoengecorrigeerd_21022025_095354.csv',
sep=';', header=3)
LaborParticipation = LaborParticipation.drop([0])
LaborParticipation.head(5)
LaborParticipation_daily = expand_quarter_to_daily(LaborParticipation, 'Unnamed: 0')
LaborParticipation_daily.head(5)



,Unnamed: 0,Beroepsbevolking|werkloze beroepsbevolking|Werkloze beroepsbevolking,Beroepsbevolking|werkloze beroepsbevolking|Kortdurig,Beroepsbevolking|werkloze beroepsbevolking|Langdurig of onbekend,Beroepsbevolking|Werkloosheidspercentage,date
2013-01-01,2013 1e kwartaal,709,477,232,"7,7",2013-01-01
2013-01-02,2013 1e kwartaal,709,477,232,"7,7",2013-01-02
2013-01-03,2013 1e kwartaal,709,477,232,"7,7",2013-01-03
2013-01-04,2013 1e kwartaal,709,477,232,"7,7",2013-01-04
2013-01-05,2013 1e kwartaal,709,477,232,"7,7",2013-01-05


In [68]:
LaborParticipation['Unnamed: 0']

1     2013 1e kwartaal
2     2013 2e kwartaal
3     2013 3e kwartaal
4     2013 4e kwartaal
5                 2013
            ...       
57    2024 2e kwartaal
58    2024 3e kwartaal
59    2024 4e kwartaal
60                2024
61           Bron: CBS
Name: Unnamed: 0, Length: 61, dtype: object

In [141]:
ConsumerPriceIndex = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Consumentenprijzen__prijsindex_2015_100_21022025_095130.csv',sep=';', header=3)
ConsumerPriceIndex.drop([0])
ConsumerPriceIndex.head(5)
ConsumerPriceIndex_daily = expand_year_to_daily(ConsumerPriceIndex, 'Unnamed: 0')
ConsumerPriceIndex_daily.head(5)

,Unnamed: 0,CPI,CPI afgeleid,Jaarmutatie CPI,Jaarmutatie CPI afgeleid,date
1996-01-01,1996,"68,94","73,25",NaN,NaN,1996-01-01
1996-01-02,1996,"68,94","73,25",NaN,NaN,1996-01-02
1996-01-03,1996,"68,94","73,25",NaN,NaN,1996-01-03
1996-01-04,1996,"68,94","73,25",NaN,NaN,1996-01-04
1996-01-05,1996,"68,94","73,25",NaN,NaN,1996-01-05


In [142]:
ConsumerPriceIndex = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Consumentenprijzen__prijsindex_2015_100_21022025_095130.csv',sep=';', header=3)
ConsumerPriceIndex.head(5)

,Unnamed: 0,CPI,CPI afgeleid,Jaarmutatie CPI,Jaarmutatie CPI afgeleid
0,Perioden,2015 = 100,2015 = 100,%,%
1,1996,"68,94","73,25",NaN,NaN
2,1997,"70,40","74,63","2,1","1,9"
3,1998,"71,78","75,93","2,0","1,7"
4,1999,"73,32","77,19","2,1","1,7"


In [143]:
PopulationSize = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bevolking_op_1_januari_en_gemiddeld__geslacht__leeftijd_en_regio_21022025_094947.csv',sep=';', header=3)
PopulationSize.head(5)

,Unnamed: 0,Onderwerp,Bevolking op 1 januari,Bevolking op 1 januari.1,Bevolking op 1 januari.2,Bevolking op 1 januari.3,Bevolking op 1 januari.4,Bevolking op 1 januari.5,Bevolking op 1 januari.6,Bevolking op 1 januari.7,...,Gemiddelde bevolking .81,Gemiddelde bevolking .82,Gemiddelde bevolking .83,Gemiddelde bevolking .84,Gemiddelde bevolking .85,Gemiddelde bevolking .86,Gemiddelde bevolking .87,Gemiddelde bevolking .88,Gemiddelde bevolking .89,Gemiddelde bevolking .90
0,NaN,Geslacht,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,...,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen,Totaal mannen en vrouwen
1,NaN,Burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,...,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat,Totaal burgerlijke staat
2,NaN,Leeftijd,Totaal,10 jaar,11 jaar,12 jaar,13 jaar,14 jaar,15 jaar,16 jaar,...,90 jaar,91 jaar,92 jaar,93 jaar,94 jaar,95 jaar,96 jaar,97 jaar,98 jaar,99 jaar
3,Regio's,Perioden,aantal,aantal,aantal,aantal,aantal,aantal,aantal,aantal,...,aantal,aantal,aantal,aantal,aantal,aantal,aantal,aantal,aantal,aantal
4,Nederland,1988,14714948,176401,180149,182392,191976,201130,220650,234758,...,.,.,.,.,.,.,.,.,.,.


In [145]:
Gdp = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Opbouw_binnenlands_product__bbp___nationale_rekeningen__1995_2022_21022025_094218.csv',sep=';', header=3).T


Gdp.columns = Gdp.iloc[0]
Gdp.head(5)
Gdp_daily = expand_year_to_daily(Gdp,use_index=True )
Gdp_daily.head(5)

Onderwerp,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Totaal,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Lonen,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Sociale premies t.l.v. werkgevers,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Saldo,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Belastingen op productie en invoer,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Subsidies,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Bruto,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Verbruik van vaste activa (-),Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Netto,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Bruto binnenlands product,...,Bbp vanuit de inkomensvorming|Prijsindexcijfers|Beloning van werknemers|Sociale premies t.l.v. werkgevers,Bbp vanuit de inkomensvorming|Prijsindexcijfers|Belastingen en subsidies|Saldo,Bbp vanuit de inkomensvorming|Prijsindexcijfers|Belastingen en subsidies|Belastingen op productie en invoer,Bbp vanuit de inkomensvorming|Prijsindexcijfers|Belastingen en subsidies|Subsidies,Bbp vanuit de inkomensvorming|Prijsindexcijfers|Exploitatieoverschot|Bruto,Bbp vanuit de inkomensvorming|Prijsindexcijfers|Exploitatieoverschot|Verbruik van vaste activa (-),Bbp vanuit de inkomensvorming|Prijsindexcijfers|Exploitatieoverschot|Netto,Bbp vanuit de inkomensvorming|Prijsindexcijfers|Bruto binnenlands product,Bron: CBS,date
1995-01-01,165485,139141,26344,31178,35959,4781,132884,52442,80442,329547,...,.,"55,0","58,8","90,7",.,"76,5",.,"70,2",NaN,1995-01-01
1995-01-02,165485,139141,26344,31178,35959,4781,132884,52442,80442,329547,...,.,"55,0","58,8","90,7",.,"76,5",.,"70,2",NaN,1995-01-02
1995-01-03,165485,139141,26344,31178,35959,4781,132884,52442,80442,329547,...,.,"55,0","58,8","90,7",.,"76,5",.,"70,2",NaN,1995-01-03
1995-01-04,165485,139141,26344,31178,35959,4781,132884,52442,80442,329547,...,.,"55,0","58,8","90,7",.,"76,5",.,"70,2",NaN,1995-01-04
1995-01-05,165485,139141,26344,31178,35959,4781,132884,52442,80442,329547,...,.,"55,0","58,8","90,7",.,"76,5",.,"70,2",NaN,1995-01-05


In [64]:
GdpProductionAndSpending = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bbp__productie_en_bestedingen__kwartalen__mutaties__na__1995__2024_I_21022025_094045.csv',sep=';', header=3).T
GdpProductionAndSpending = GdpProductionAndSpending.reset_index(drop=True)
# set header of
GdpProductionAndSpending.columns = GdpProductionAndSpending.iloc[0]
GdpProductionAndSpending = GdpProductionAndSpending.drop([0,1])
GdpProductionAndSpending = GdpProductionAndSpending.drop('Bron: CBS', axis=1)   
GdpProductionAndSpending_daily = expand_quarter_to_daily(GdpProductionAndSpending, 'Onderwerp')
GdpProductionAndSpending_daily[100::105]


,Onderwerp,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Bruto binnenlands product,"Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Bbp, gecorrigeerd voor werkdageneffecten",Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Invoer van goederen en diensten|Totaal,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Invoer van goederen en diensten|Goederen,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Invoer van goederen en diensten|Diensten,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Consumptieve bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Consumptieve bestedingen|Huishoudens,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Consumptieve bestedingen|Overheid,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Bruto investeringen in vaste activa|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Bruto investeringen in vaste activa|Bedrijven en huishoudens,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Bruto investeringen in vaste activa|Overheid,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Verandering in voorraden,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Uitvoer van goederen en diensten|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Uitvoer van goederen en diensten|Goederen,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Uitvoer van goederen en diensten|Diensten,date
1995-04-11,1995 2e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1995-04-11
1995-07-25,1995 3e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1995-07-25
1995-11-07,1995 4e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1995-11-07
1996-02-20,1996 1e kwartaal,"3,5","2,4",.,"5,6","5,1","7,9","3,5","2,8","3,0","5,3","-1,8","0,0","0,7","-3,7","0,1","4,6","4,0","7,1",1996-02-20
1996-06-04,1996 2e kwartaal,"3,4","4,0",.,"2,4","1,2","6,8","3,4","5,3","2,9","5,0","-1,6","5,9","5,1","10,2","0,0","0,4","-1,2","6,8",1996-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-09,2019 4e kwartaal,"0,9","2,7",.,"-1,4","-2,6","2,0","0,9","2,6","2,5","2,5","2,5","2,7","2,8","2,1",.,"-0,8","-1,7","1,8",2019-10-09
2020-01-22,2020 1e kwartaal,"0,5","1,9",.,"-1,5","-2,7","1,8","0,5","2,4","1,9","1,4","2,6","3,3","3,5","2,2",.,"-1,6","-2,6","1,2",2020-01-22
2020-05-06,2020 2e kwartaal,"-1,3","1,6",.,"-5,4","-7,8","1,2","-1,3","1,5","1,4","1,5","1,2","2,7","3,0","1,3",.,"-4,5","-6,5","1,0",2020-05-06
2020-08-19,2020 3e kwartaal,"-0,7","2,0",.,"-4,3","-5,9","0,4","-0,7","1,6","1,4","0,9","2,2","2,2","2,5","0,9",.,"-3,1","-4,5","1,0",2020-08-19
